# Facial Landmark Detection - Intro to Deep Learning

### Import and Augment Data

In [1]:
# General imports
import pandas as pd
import numpy as np
import torch
from PIL import Image

#### Download & Import Data

In [3]:
# Download data from GitHub
!pip install wget -qU
import wget
import os.path

if not os.path.isfile("./facial-keypoints-detection.zip"):
    url = r"https://github.com/ko-redtruck/facial-landmark-detection/raw/main/facial-keypoints-detection.zip"
    wget.download(url, ".")

In [ ]:
# Read training data
import zipfile

DATA_DIR = "./data"

def load_data_and_filter_from(file: str):
    data = pd.read_csv(f'{DATA_DIR}/{file}')
    data = data.dropna()
    return data

with zipfile.ZipFile("./facial-keypoints-detection.zip", 'r') as zip_ref:
    zip_ref.extractall("data")

training_data = load_data_and_filter_from('training.zip')
# training_data

In [ ]:
# Convert data to numpy arrays

def convert_to_numpy(data: pd.DataFrame):
    X = [np.fromstring(image, dtype=np.uint8, sep=' ').reshape(96,96) for image in data["Image"]]
    X = np.reshape(X,(-1,96, 96))

    Y = np.delete(data.values,30,axis=1)
    Y = Y.reshape(-1,30).astype('float32')
    return X,Y

np_images, labels = convert_to_numpy(training_data)
np_images.shape, labels.shape

In [ ]:
# Plot images
import matplotlib.pyplot as plt
from torchvision.transforms.functional import to_pil_image

def plot_image(image: Image, labeling: np.ndarray=None):
    try:
        image = to_pil_image(image)
    finally:
        plt.imshow(image, interpolation='nearest', cmap='gray')
        if labeling is not None:
            print(len(labeling))
            for i in range(0, len(labeling)-1, 2):
                plt.plot(labeling[i + 0],labeling[i + 1], marker=".", color='cyan')
        plt.show()

def plot_images(images: np.ndarray, labels: np.ndarray=None, num: int=None):
    for i in range(num if num is not None else len(images)):
        plot_image(images[i], labels[i])

In [ ]:
plot_images(np_images, labels, 3)

#### Image Preparation

In [ ]:
# Transformation and augmentation of images
from torchvision import transforms
import torchvision.transforms.functional as tf
from numpy.random import randint, random

def transform_image(image: np.ndarray):
    transform = transforms.Compose([
        transforms.ColorJitter(0.5, 0.5, 0.5, 0.5),
        transforms.RandomErasing(scale=(0.01, 0.02))
    ])

    image = tf.to_tensor(image)
    image = tf.adjust_sharpness(image, 2 * random())
    return transform(image)

def random_resize_datapoint(image: torch.Tensor, labeling: np.ndarray):
    new_size = randint(image.shape[-1], 224, dtype='uint8')
    scaling = new_size / image.shape[-1]

    return tf.resize(image, [new_size, new_size]), labeling * scaling

def augment_data(images, label_list, size_factor: int=1):
    augmented_images = []
    augmented_labels = []

    for _ in range(size_factor):
        for i in range(len(images)):
            image = transform_image(images[i])
            image, labeling = random_resize_datapoint(image, label_list[i])

            augmented_images.append(image)
            augmented_labels.append(labeling)

    return augmented_images, augmented_labels

In [ ]:
DATASET_AUGMENTATION_FACTOR = 2

augmented_images, augmented_labels = augment_data(np_images, labels, DATASET_AUGMENTATION_FACTOR)
plot_images(augmented_images, augmented_labels, 3)

In [ ]:
# Padding images
from numpy.random import randint

def random_pad_image_with_noise(image: torch.Tensor, new_size: (int, int), offset_x: int=0, offset_y: int=0):
    random_image = tf.to_tensor(randint(256, dtype='uint8', size=new_size))

    random_image[:, offset_y:image.shape[-2] + offset_y, offset_x:image.shape[-1] + offset_x] = image
    return random_image

def random_offset(image_dimensions, new_size: (int, int)):
    min_offset = (0, 0)
    max_offset = np.subtract(new_size, image_dimensions)

    offset_y = randint(min_offset[0], max_offset[0])
    offset_x = randint(min_offset[1], max_offset[1])
    return offset_y, offset_x

def random_pad_datapoint(image: torch.Tensor, labeling: np.ndarray, new_size: (int, int)):
    offset_y, offset_x = random_offset(image.shape[-2:], new_size)

    padded_image = random_pad_image_with_noise(image, new_size, offset_x=offset_x, offset_y=offset_y)
    adjusted_labeling = np.zeros(labeling.shape, dtype=labeling.dtype)
    adjusted_labeling[:-1:2] = labeling[:-1:2] + offset_x
    adjusted_labeling[1::2] = labeling[1::2] + offset_y

    return padded_image, adjusted_labeling

def random_pad_data(images, labels, new_size: (int, int)):
    padded_images = []
    adjusted_labels = []

    for i in range(len(images)):
        image, labeling = random_pad_datapoint(image=images[i], labeling=labels[i], new_size=new_size)
        padded_images.append(image)
        adjusted_labels.append(labeling)

    return padded_images, adjusted_labels

In [ ]:
padded_images, final_labels = random_pad_data(augmented_images, augmented_labels, (224, 224))
plot_images(padded_images, final_labels, 3)

In [ ]:
# Transform images to RGB
final_images = torch.zeros((len(padded_images), 3, 224, 224))
final_images[:, :] = padded_images[:][0]
final_images.shape

### Training
#### Set up wandb & Cuda device

In [ ]:
#Instantiating CUDA device
import torch

def device():
    return torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device()

In [ ]:
!pip install wandb -qU
import wandb

wandb.login()

#### Define model and parameters for run

In [ ]:
NET = "ResNet18"
FC_LAYER = "Lin-ReLu-Lin"
OPTIMIZER = "Adam"
LOSS = "MSE"
EPOCHS = 200
BATCH_SIZE = 400
LR = 0.001
MOMENTUM = 0.9 # SGD
WEIGHT_DECAY = 0.01

In [ ]:
# Define parameter mappings
from torchvision.models import resnet18
from torch.optim import Adam
from torch import nn

def add_fc(net, layer):
    net.fc = layer
    return net

fc_layers = {
    "Lin-ReLu-Lin": nn.Sequential(
        nn.Linear(512,256),
        nn.ReLU(),
        nn.Linear(256,30)
    ),
    "Linear": nn.Linear(512, 30)
}

networks = {
    "ResNet18": add_fc(resnet18(pretrained=True), fc_layers[FC_LAYER])
}

optimizers = {
    "Adam": Adam(networks[NET].parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
}

loss_functions = {
    "MSE": nn.MSELoss()
}

#### Set up Training

In [ ]:
import sys

net = networks[NET].to(device())
optimizer = optimizers[OPTIMIZER]
loss_function = loss_functions[LOSS]
num_workers = 0 if sys.platform.startswith('win') else 2

In [ ]:
# Define data loaders
from torch.utils.data import DataLoader, random_split

def get_data_loaders(images, labels, batch_size, test_data_split=0.1, num_workers=2):
    dataset = list(zip(images, labels))
    training_data_size = int(len(dataset) * (1-test_data_split))
    train, test = random_split(dataset, [training_data_size, len(dataset) - training_data_size])

    train_loader = DataLoader(train, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    test_loader = DataLoader(test, batch_size=batch_size, num_workers=num_workers)
    return train_loader,test_loader

In [ ]:
train_loader, test_loader = get_data_loaders(final_images, final_labels, batch_size=BATCH_SIZE, test_data_split=0.15, num_workers=num_workers)

#### Training

In [ ]:
# Define training and testing steps

def compute_loss(inputs, labels, net, loss_function):
    outputs = net(inputs.to(device()))
    labels = labels.to(device())

    return loss_function(outputs, labels)

def evaluate_model_on_dataset(net, loss_function, data_loader, optimize_weights=None):
    avg_loss = 0.

    for images, labels in data_loader:
        loss = compute_loss(images, labels, net, loss_function)
        avg_loss += loss

        if optimize_weights is not None:
            optimize_weights(loss)

    return avg_loss / len(data_loader)

def optimize_weights(optimizer):
    def optimize_weights(loss):
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    return optimize_weights

In [ ]:
# Init wandb tracking
wandb.init(project="facial-landmark-detection", entity="leo-team", config={"net": NET, "optimizer": OPTIMIZER, "final_layer": FC_LAYER, "epochs": EPOCHS, "batch_size": BATCH_SIZE, "lr": LR, "momentum": MOMENTUM, "weight decay" : WEIGHT_DECAY, "" : DATASET_AUGMENTATION_FACTOR})

# Train the model
for epoch in range(EPOCHS): # loop over dataset several times
    net.train()
    train_loss = evaluate_model_on_dataset(net, loss_function, train_loader, optimize_weights(optimizer))

    net.eval()
    test_loss = evaluate_model_on_dataset(net, loss_function, test_loader)

    # log error
    print('epoch {}, train loss {}, test loss {}'.format(epoch+1, train_loss, test_loss))
    wandb.log({
        "train_loss": train_loss,
        "test_loss": test_loss,
    })